<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


Objetivos
Crear un archivo web de los registros de lanzamiento de Falcon 9 con BeautifulSoup:

Extraer una tabla HTML de registros de lanzamiento de Falcon 9 de Wikipedia.
Analizar la tabla y convertirla en un marco de datos de Pandas.
Primero, importemos los paquetes necesarios para este laboratorio.

First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [6]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [7]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [8]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the List of Falcon 9 and Falcon Heavy launches Wikipage updated on 9th June 2021

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [9]:
# use requests.get() method with the provided static_url
# assign the response to a object

response = requests.get(static_url)

# Comprobar el código de estado para asegurarte de que la solicitud fue exitosa
print("Status code:", response.status_code)


Status code: 200


Create a `BeautifulSoup` object from the HTML `response`


In [11]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

from bs4 import BeautifulSoup

# Crear el objeto BeautifulSoup a partir del contenido de la respuesta
soup = BeautifulSoup(response.text, 'html.parser')

# Verificación rápida: mostrar el título de la página
print(soup.title)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
# Imprimir el título de la página
print(soup.title)


<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [19]:
# Use the find_all function in the BeautifulSoup object, with element type `table`

# TAREA 2: Extraer todas las tablas HTML y asignarlas a html_tables
html_tables = soup.find_all('table')

# Mostrar cuántas tablas se encontraron
print("Número de tablas encontradas:", len(html_tables))


# Assign the result to a list called `html_tables`

# Buscar todos los elementos <table> en el HTML
#html_tables = soup.find_all('table')



Número de tablas encontradas: 25


In [13]:
import requests
import pandas as pd

# Realiza la solicitud GET
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)
data = response.json()

# Convierte la respuesta a un DataFrame
df = pd.json_normalize(data)

# Asegúrate de que la columna 'static_fire_date_utc' esté en formato de fecha
df['static_fire_date_utc'] = pd.to_datetime(df['static_fire_date_utc'])

# Extrae el año de la primera fila de la columna 'static_fire_date_utc'
first_year = df.loc[0, 'static_fire_date_utc'].year

print(first_year)

2006


In [40]:
for i, table in enumerate(html_tables):
    print(f"Tabla {i} - clases: {table.get('class')}")


Tabla 0 - clases: ['col-begin']
Tabla 1 - clases: ['col-begin']
Tabla 2 - clases: ['wikitable', 'plainrowheaders', 'collapsible']
Tabla 3 - clases: ['wikitable', 'plainrowheaders', 'collapsible']
Tabla 4 - clases: ['wikitable', 'plainrowheaders', 'collapsible']
Tabla 5 - clases: ['wikitable', 'plainrowheaders', 'collapsible']
Tabla 6 - clases: ['wikitable', 'plainrowheaders', 'collapsible']
Tabla 7 - clases: ['wikitable', 'plainrowheaders', 'collapsible']
Tabla 8 - clases: ['wikitable', 'plainrowheaders', 'collapsible']
Tabla 9 - clases: ['wikitable', 'plainrowheaders', 'collapsible']
Tabla 10 - clases: ['wikitable', 'plainrowheaders', 'collapsible']
Tabla 11 - clases: ['wikitable']
Tabla 12 - clases: ['wikitable']
Tabla 13 - clases: ['wikitable']
Tabla 14 - clases: ['wikitable']
Tabla 15 - clases: ['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
Tabla 16 - clases: ['nowraplinks', 'navbox-subgroup']
Tabla 17 - clases: ['nowraplinks', 'navbox-subgroup']
Tabla 1

In [42]:
first_launch_table = html_tables[2]
headers = [extract_column_from_header(th) for th in first_launch_table.find_all('th')]
print(headers)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


In [16]:
#print(data[:2])  # Imprime las dos primeras entradas para inspección

Starting from the third table is our target table contains the actual launch records.


In [17]:
# Let's print the third table and check its content
#first_launch_table = html_tables[2]
#print(first_launch_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [21]:
# Ya tienes html_tables
# Vamos a inspeccionar las primeras tablas para identificar la correcta
for i, table in enumerate(html_tables[:5]):
    print(f"Tabla {i}")
    print(table.prettify()[:500])  # Muestra los primeros 500 caracteres de cada tabla
    print("-" * 80)


Tabla 0
<table class="col-begin" role="presentation">
 <tbody>
  <tr>
   <td class="col-break">
    <div class="mw-heading mw-heading3">
     <h3 id="Rocket_configurations">
      Rocket configurations
     </h3>
    </div>
    <div class="chart noresize" style="padding-top:10px;margin-top:1em;max-width:420px;">
     <div style="position:relative;min-height:320px;min-width:420px;max-width:420px;">
      <div style="float:right;position:relative;min-height:240px;min-width:320px;max-width:320px;border-lef
--------------------------------------------------------------------------------
Tabla 1
<table class="col-begin" role="presentation">
 <tbody>
  <tr>
   <td class="col-break">
    <div class="mw-heading mw-heading3">
     <h3 id="Launch_outcomes">
      Launch outcomes
     </h3>
    </div>
    <div class="chart noresize" style="padding-top:10px;margin-top:1em;max-width:480px;">
     <div style="position:relative;min-height:320px;min-width:480px;max-width:480px;">
      <div style="flo

In [23]:
# Supongamos que la tabla correcta es la número 2 (ajústalo según lo que veas)
first_launch_table = html_tables[2]


In [49]:
column_names = []

def extract_column_from_header(th):
    return th.get_text(strip=True)

table_headers = first_launch_table.find_all('th')

for th in table_headers:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

print("Columnas extraídas:")
print(column_names)


Columnas extraídas:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


In [50]:
#df[df['rocket.rocket_name'] == 'Falcon 9']

In [51]:
df['launchpad'].isnull().sum()

KeyError: 'launchpad'

Check the extracted column names


In [27]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [29]:
# Crear diccionario vacío con claves de los nombres de columna
launch_dict = dict.fromkeys(column_names)

# Inicializar cada valor como una lista vacía
for key in launch_dict.keys():
    launch_dict[key] = []

# (Opcional) Eliminar columnas irrelevantes por nombre exacto si hace falta
# Por ejemplo, si decides que la columna "1", "2", etc. no es relevante:
for col in ['1', '2', '3', '4', '5', '6', '7']:
    if col in launch_dict:
        del launch_dict[col]

# Crear columnas personalizadas para mejor estructura
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Mostrar claves actuales del diccionario (verificación)
print("Claves del diccionario launch_dict:")
print(launch_dict.keys())


Claves del diccionario launch_dict:
dict_keys(['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', 'Version Booster', 'Booster landing', 'Date', 'Time'])


In [34]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

KeyError: 'Date and time ( )'

In [46]:
launch_dict = {
    'Flight No.': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Version Booster': [],
    'Booster landing': [],
    'Date': [],
    'Time': []
}


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [47]:
extracted_row = 0

# Extraer cada tabla con clase específica
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Obtener cada fila
    for rows in table.find_all("tr"):
        # Verificar si el encabezado de fila es un número de vuelo
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        # Obtener columnas de datos
        row = rows.find_all('td')

        if flag and row:
            extracted_row += 1

            # Flight Number
            launch_dict['Flight No.'].append(flight_number)

            # Fecha y hora
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',') if len(datatimelist) > 0 else ''
            time = datatimelist[1] if len(datatimelist) > 1 else ''
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # Booster version
            bv = booster_version(row[1])
            if not bv and row[1].a:
                bv = row[1].a.string.strip()
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string.strip() if row[2].a else row[2].get_text(strip=True)
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string.strip() if row[3].a else row[3].get_text(strip=True)
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string.strip() if row[5].a else row[5].get_text(strip=True)
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string.strip() if row[6].a else row[6].get_text(strip=True)
            launch_dict['Customer'].append(customer)

            # Launch outcome
            launch_outcome = list(row[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)


AttributeError: 'NoneType' object has no attribute 'strip'

In [48]:
df['static_fire_date_utc'].iloc[0]

KeyError: 'static_fire_date_utc'

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [43]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

df.value_counts()

Flight No.  Launch site     Payload                               Payload mass  Orbit        Customer  Launch outcome  Version Booster   Booster landing  Date               Time 
1           CCAFS           Dragon Spacecraft Qualification Unit  0             LEO          SpaceX    Success         F9 v1.07B0003.18  Failure          4 June 2010        18:45    1
2           CCAFS           Dragon                                0             LEO          NASA      Success         F9 v1.07B0004.18  Failure          8 December 2010    15:43    1
8           CCAFS           Thaicom 6                             3,325 kg      GTO          Thaicom   Success         F9 v1.1           No attempt       6 January 2014     22:06    1
7           CCAFS           SES-8                                 3,170 kg      GTO          SES       Success         F9 v1.1           No attempt       3 December 2013    22:41    1
6           VAFB            CASSIOPE                              500 kg        Polar

In [44]:
# Filtrar lanzamientos que no sean Falcon 1
falcon9_launches = df_launches[~df_launches['Version Booster'].str.contains('Falcon 1', case=False, na=False)]

# Contar la cantidad de lanzamientos Falcon 9
num_falcon9 = falcon9_launches.shape[0]

print(f"Número de lanzamientos Falcon 9 (excluyendo Falcon 1): {num_falcon9}")


NameError: name 'df_launches' is not defined

In [45]:
# Mostrar la cantidad de datos faltantes por cada columna
missing_values = df_launches.isnull().sum()

print(missing_values)


NameError: name 'df_launches' is not defined

Al final del proceso de recogida de datos de la API , ¿cuántos valores que faltan hay para la columna landingPad?

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
